In [4]:
!pip install torch numpy scipy seaborn pandas scikit-learn matplotlib tqdm

  Using cached torch-2.0.1-cp310-none-macosx_11_0_arm64.whl (55.8 MB)
  Obtaining dependency information for numpy from https://files.pythonhosted.org/packages/c3/ea/1d95b399078ecaa7b5d791e1fdbb3aee272077d9fd5fb499593c87dec5ea/numpy-1.25.2-cp310-cp310-macosx_11_0_arm64.whl.metadata
  Using cached numpy-1.25.2-cp310-cp310-macosx_11_0_arm64.whl.metadata (5.6 kB)
  Obtaining dependency information for scipy from https://files.pythonhosted.org/packages/69/9d/c6877258056c94027cae871fb5bc220edbaab5be2d6f6bc7e6081164b038/scipy-1.11.1-cp310-cp310-macosx_12_0_arm64.whl.metadata
  Using cached scipy-1.11.1-cp310-cp310-macosx_12_0_arm64.whl.metadata (54 kB)
  Using cached seaborn-0.12.2-py3-none-any.whl (293 kB)
  Obtaining dependency information for pandas from https://files.pythonhosted.org/packages/4a/f6/f620ca62365d83e663a255a41b08d2fc2eaf304e0b8b21bb6d62a7390fe3/pandas-2.0.3-cp310-cp310-macosx_11_0_arm64.whl.metadata
  Using cached pandas-2.0.3-cp310-cp310-macosx_11_0_arm64.whl.metadata (18 

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import scipy
import random

In [6]:
import pandas as pd
data_clean= pd.read_csv("data_train_clean_onset.csv")


#increment label by 0.1
#data_clean['Label'] = data_clean['Label'] + 0.1


In [7]:
# Create a list of all unique subjects
    
subjects = data_clean['Sbj_ID'].unique()

# Randomly select some of the 80% of subjects to be in the training set, 20% to the validation set and 20% in the test set
np.random.seed(42)


train_subjects = np.random.choice(subjects, size=int(0.8*len(subjects)), replace=False)
test_subjects = np.setdiff1d(subjects, train_subjects)

In [9]:

train_data = data_clean[data_clean["Sbj_ID"].isin(train_subjects)]
test_data = data_clean[data_clean["Sbj_ID"].isin(test_subjects)]

# Drop non-feature columns
train_features = train_data.drop(columns=["Sbj_ID", "Label", "ifCleanOnset", "Time2Sleep", "SleepStage"])
test_features = test_data.drop(columns=["Sbj_ID", "Label", "ifCleanOnset", "Time2Sleep", "SleepStage"])


In [10]:
test_features

,Age,Gender,Race1,Race2,Race3,Race4,delta,theta,alpha,beta,...,wrseltr5,slpapnea5,cpap5,dntaldv5,uvula5,insmnia5,rstlesslgs5,whiirs5c,epslpscl5c,hoostmeq5c
300,61,1,0.0,0.0,0.0,1.0,-1.724410,-6.903817,-5.770776,-6.237482,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,1.0,10.0
301,61,1,0.0,0.0,0.0,1.0,0.496236,-0.083275,-0.110885,0.378090,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,1.0,10.0
302,61,1,0.0,0.0,0.0,1.0,-0.650443,-1.298275,-0.121725,-0.122548,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,1.0,10.0
303,61,1,0.0,0.0,0.0,1.0,-0.498544,-1.553040,-1.672716,-0.844400,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,1.0,10.0
304,61,1,0.0,0.0,0.0,1.0,0.341871,0.239111,0.166970,1.024611,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,1.0,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92695,72,1,1.0,0.0,0.0,0.0,2.458438,2.316242,2.077480,-0.054839,...,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,13.0,11.0,15.0
92696,72,1,1.0,0.0,0.0,0.0,2.363469,2.166921,1.700581,-0.250445,...,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,13.0,11.0,15.0
92697,72,1,1.0,0.0,0.0,0.0,3.217854,2.529125,2.155246,-0.116003,...,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,13.0,11.0,15.0
92698,72,1,1.0,0.0,0.0,0.0,2.676446,2.478466,2.226313,0.149029,...,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,13.0,11.0,15.0


In [16]:
!pip install kmm

In [21]:
!pip install densratio

In [11]:
data_clean.columns

Index(['Label', 'Sbj_ID', 'Age', 'Gender', 'Race1', 'Race2', 'Race3', 'Race4',
       'ifCleanOnset', 'Time2Sleep', 'SleepStage', 'delta', 'theta', 'alpha',
       'beta', 'DTratio', 'DAratio', 'DBratio', 'TAratio', 'TBratio',
       'ABratio', 'Transratio', 'deltapeak', 'thetapeak', 'alphapeak',
       'betapeak', 'TCdelta', 'TCtheta', 'TCalpha', 'TCbeta',
       'DN_HistogramMode_5', 'DN_HistogramMode_10', 'CO_f1ecac',
       'CO_FirstMin_ac', 'CO_HistogramAMI_even_2_5', 'CO_trev_1_num',
       'MD_hrv_classic_pnn40', 'SB_BinaryStats_mean_longstretch1',
       'SB_TransitionMatrix_3ac_sumdiagcov', 'PD_PeriodicityWang_th0_01',
       'CO_Embed2_Dist_tau_d_expfit_meandiff',
       'IN_AutoMutualInfoStats_40_gaussian_fmmi',
       'FC_LocalSimple_mean1_tauresrat', 'DN_OutlierInclude_p_001_mdrmd',
       'DN_OutlierInclude_n_001_mdrmd', 'SP_Summaries_welch_rect_area_5_1',
       'SB_BinaryStats_diff_longstretch0', 'SB_MotifThree_quantile_hh',
       'SC_FluctAnal_2_rsrangefit_50_1_logi_p

In [12]:
from densratio import densratio
columns_to_remove = ['Sbj_ID', 'Label', 'ifCleanOnset', 'Time2Sleep', 'SleepStage', 'Race1', 'Race2', 'Race3', 'Race4', 'nap5', 'trbleslpng5', 'slpngpills5',
       'sleepy5', 'legsdscmfrt5', 'rubbnglgs5', 'wrserest5', 'wrseltr5',
       'slpapnea5', 'cpap5', 'dntaldv5', 'uvula5', 'insmnia5', 'rstlesslgs5',
       'whiirs5c', 'epslpscl5c', 'hoostmeq5c', 'Gender', 'O-information', 'S-information']
columns_left = ['Label', 'delta', 'theta', 'alpha',
       'beta', 'DTratio', 'DAratio', 'DBratio', 'TAratio', 'TBratio',
       'ABratio', 'Transratio', 'deltapeak', 'thetapeak', 'alphapeak',
       'betapeak', 'TCdelta', 'TCtheta', 'TCalpha', 'TCbeta',
       'DN_HistogramMode_5', 'DN_HistogramMode_10', 'CO_f1ecac',
       'CO_FirstMin_ac', 'CO_HistogramAMI_even_2_5', 'CO_trev_1_num',
       'MD_hrv_classic_pnn40', 'SB_BinaryStats_mean_longstretch1',
       'SB_TransitionMatrix_3ac_sumdiagcov', 'PD_PeriodicityWang_th0_01',
       'CO_Embed2_Dist_tau_d_expfit_meandiff',
       'IN_AutoMutualInfoStats_40_gaussian_fmmi',
       'FC_LocalSimple_mean1_tauresrat', 'DN_OutlierInclude_p_001_mdrmd',
       'DN_OutlierInclude_n_001_mdrmd', 'SP_Summaries_welch_rect_area_5_1',
       'SB_BinaryStats_diff_longstretch0', 'SB_MotifThree_quantile_hh',
       'SC_FluctAnal_2_rsrangefit_50_1_logi_prop_r1',
       'SC_FluctAnal_2_dfa_50_1_2_logi_prop_r1',
       'SP_Summaries_welch_rect_centroid', 'FC_LocalSimple_mean3_stderr',
       'gamma', 'GAration', 'GBratio', 'GTratio', 'GDratio', 'gammapeak',
       'TCgamma', 'DTcouple', 'DAcouple', 'DBcouple', 'TAcouple', 'TBcouple',
       'ABcouple', 'DGcouple', 'TGcouple', 'AGcouple', 'BGcouple',
       'Entropy Rate LZ', 'Aperiodic component (spectral exponent)']

unique_timepoints = data_clean['Label'].unique()

# Dictionary to store weights for each timepoint
# DataFrames to store variances
variances_df_train = pd.DataFrame()
variances_df_test = pd.DataFrame()

for timepoint in unique_timepoints:
    # Filter data for the current timepoint
    train_timepoint_data = train_data[train_data['Label'] == timepoint].drop(columns=columns_to_remove)
    test_timepoint_data = test_data[test_data['Label'] == timepoint].drop(columns=columns_to_remove)

    train_variance = train_timepoint_data.var().to_frame().transpose()
    test_variance = test_timepoint_data.var().to_frame().transpose()

    # Create a new row for each dataset with timepoint label and variances
    train_row = pd.concat([pd.DataFrame({'Label': [timepoint]}), train_variance], axis=1)
    test_row = pd.concat([pd.DataFrame({'Label': [timepoint]}), test_variance], axis=1)

    # Append these rows to the main DataFrames using pd.concat
    variances_df_train = pd.concat([variances_df_train, train_row], ignore_index=True)
    variances_df_test = pd.concat([variances_df_test, test_row], ignore_index=True)

# Displaying the test dataframe
print(variances_df_test)
    


     Label        Age     delta     theta     alpha      beta   DTratio  \
0     29.9  82.901111  1.146668  2.844785  2.117829  2.946534  1.783798   
1     29.8  82.901111  1.707846  1.480746  1.618907  1.120598  1.022546   
2     29.7  82.901111  1.307333  0.965980  1.012678  0.531080  0.789811   
3     29.6  82.901111  1.862556  2.142097  2.135987  2.319839  2.086731   
4     29.5  82.901111  1.266521  1.082494  1.270436  1.850224  1.157423   
..     ...        ...       ...       ...       ...       ...       ...   
295    0.4  82.901111  4.946456  1.589600  0.964305  0.723038  1.956304   
296    0.3  82.901111  4.654417  1.684920  1.336623  1.408039  1.494094   
297    0.2  82.901111  5.364773  2.053983  0.933794  0.916510  1.362559   
298    0.1  82.901111  7.088137  2.172118  1.043118  1.179020  1.612214   
299    0.0  82.901111  5.106812  1.560303  1.192096  0.977417  1.767694   

      DAratio   DBratio   TAratio  ...  DBcouple  TAcouple  TBcouple  \
0    1.544302  1.913628  1.

In [59]:
# Threshold below which variance is considered too small
threshold = 1e-1

# Identify features in the training set with variances below threshold
low_variance_features_train = variances_df_train.columns[variances_df_train.mean() < threshold]

# Identify features in the test set with variances below threshold
low_variance_features_test = variances_df_test.columns[variances_df_test.mean() < threshold]

# Combine the results to get a list of problematic features
problematic_features = set(low_variance_features_train).union(set(low_variance_features_test))

print("Problematic features:", problematic_features)

Problematic features: set()


In [14]:
zero_variance_features_train = train_data.var()[train_data.var() == 0].index.tolist()
zero_variance_features_test = test_data.var()[test_data.var() == 0].index.tolist()

zero_variance_features = set(zero_variance_features_train).union(set(zero_variance_features_test))
print("Features with zero variance:", zero_variance_features)

Features with zero variance: {'ifCleanOnset'}


In [13]:
train_data_no_nan = train_data.dropna(axis=0)
test_data_no_nan = test_data.dropna(axis=0)

In [92]:
from densratio import densratio
columns_to_remove = ['Sbj_ID', 'Age', 'Label', 'ifCleanOnset', 'Time2Sleep', 'SleepStage', 'Race1', 'Race2', 'Race3', 'Race4', 'nap5', 'trbleslpng5', 'slpngpills5',
       'sleepy5', 'legsdscmfrt5', 'rubbnglgs5', 'wrserest5', 'wrseltr5',
       'slpapnea5', 'cpap5', 'dntaldv5', 'uvula5', 'insmnia5', 'rstlesslgs5',
       'whiirs5c', 'epslpscl5c', 'hoostmeq5c', 'Gender', 'O-information', 'S-information']


unique_timepoints = data_clean['Label'].unique()

# Dictionary to store weights for each timepoint
timepoint_weights = {}

for timepoint in unique_timepoints:
    # Filter data for the current timepoint
    train_timepoint_data = train_data_no_nan[train_data_no_nan['Label'] == timepoint].drop(columns = columns_to_remove).copy()
    test_timepoint_data = test_data_no_nan[test_data_no_nan['Label'] == timepoint].drop(columns = columns_to_remove).copy()


    # Calculate weights using KMM (or KLIEP)
    #beta = kmm(train_timepoint_data.values, test_timepoint_data.values)
    print("Calculating weights for timepoint: ", timepoint)
    
    # Alternatively, for KLIEP
    result = densratio(train_timepoint_data.values, test_timepoint_data.values, verbose=False)
    beta = result.compute_density_ratio(train_timepoint_data.values)

    # Store the weights in the dictionary
    timepoint_weights[timepoint] = beta
timepoint_weights


Calculating weights for timepoint:  29.9
Calculating weights for timepoint:  29.8
Calculating weights for timepoint:  29.7
Calculating weights for timepoint:  29.6
Calculating weights for timepoint:  29.5
Calculating weights for timepoint:  29.4
Calculating weights for timepoint:  29.3
Calculating weights for timepoint:  29.2
Calculating weights for timepoint:  29.1
Calculating weights for timepoint:  29.0
Calculating weights for timepoint:  28.9
Calculating weights for timepoint:  28.8
Calculating weights for timepoint:  28.7
Calculating weights for timepoint:  28.6
Calculating weights for timepoint:  28.5
Calculating weights for timepoint:  28.4
Calculating weights for timepoint:  28.3
Calculating weights for timepoint:  28.2
Calculating weights for timepoint:  28.1
Calculating weights for timepoint:  28.0
Calculating weights for timepoint:  27.9
Calculating weights for timepoint:  27.8
Calculating weights for timepoint:  27.7
Calculating weights for timepoint:  27.6
Calculating weig

{29.9: array([0.99999, 0.99999, 0.99999, 0.99999, 0.99999, 0.99999, 0.99999,
        0.99999, 0.99999, 0.99999, 0.99999, 0.99999, 0.99999, 0.99999,
        0.99999, 0.99999, 0.99999, 0.99999, 0.99999, 0.99999, 0.99999,
        0.99999, 0.99999, 0.99999, 0.99999, 0.99999, 0.99999, 0.99999,
        0.99999, 0.99999, 0.99999, 0.99999, 0.99999, 0.99999, 0.99999,
        0.99999, 0.99999, 0.99999, 0.99999, 0.99999, 0.99999, 0.99999,
        0.99999, 0.99999, 0.99999, 0.99999, 0.99999, 0.99999, 0.99999,
        0.99999, 0.99999, 0.99999, 0.99999, 0.99999, 0.99999, 0.99999,
        0.99999, 0.99999, 0.99999, 0.99999, 0.99999, 0.99999, 0.99999,
        0.99999, 0.99999, 0.99999, 0.99999, 0.99999, 0.99999, 0.99999,
        0.99999, 0.99999, 0.99999, 0.99999, 0.99999, 0.99999, 0.99999,
        0.99999, 0.99999, 0.99999, 0.99999, 0.99999, 0.99999, 0.99999,
        0.99999, 0.99999, 0.99999, 0.99999, 0.99999, 0.99999, 0.99999,
        0.99999, 0.99999, 0.99999, 0.99999, 0.99999, 0.99999, 0.99999,


In [85]:
from densratio import densratio

columns_to_remove = ['Sbj_ID', 'Label', 'ifCleanOnset', 'Age' 'Time2Sleep', 'SleepStage', 'Race1', 'Race2', 'Race3', 'Race4', 'nap5', 'trbleslpng5', 'slpngpills5',
       'sleepy5', 'legsdscmfrt5', 'rubbnglgs5', 'wrserest5', 'wrseltr5',
       'slpapnea5', 'cpap5', 'dntaldv5', 'uvula5', 'insmnia5', 'rstlesslgs5',
       'whiirs5c', 'epslpscl5c', 'hoostmeq5c', 'Gender', 'O-information', 'S-information']

unique_timepoints = data_clean['Label'].unique()

# Threshold below which variance is considered too small
threshold = 1e-1

# Dictionary to store weights for each timepoint
timepoint_weights = {}

for timepoint in unique_timepoints:
    
    # Filter data for the current timepoint
    columns_to_drop_train = [col for col in columns_to_remove if col in train_data.columns]
    columns_to_drop_test = [col for col in columns_to_remove if col in test_data.columns]
    
    train_timepoint_data = train_data[train_data['Label'] == timepoint].drop(columns=columns_to_drop_train)
    test_timepoint_data = test_data[test_data['Label'] == timepoint].drop(columns=columns_to_drop_test)

    
    # Check for low variance features within this timepoint
    low_variance_features_train = train_timepoint_data.columns[train_timepoint_data.var() < threshold].tolist()
    low_variance_features_test = test_timepoint_data.columns[test_timepoint_data.var() < threshold].tolist()
    
    problematic_features = set(low_variance_features_train).union(set(low_variance_features_test))
    
    if problematic_features:
        print(f"Timepoint {timepoint} has problematic features: {problematic_features}")
        train_timepoint_data = train_timepoint_data.drop(columns=problematic_features)
        test_timepoint_data = test_timepoint_data.drop(columns=problematic_features)
    
    print("Timepoint:", timepoint)
    print(train_timepoint_data.values)
  
    
    # Alternatively, for KLIEP
    try:
        result = densratio(train_timepoint_data.values, test_timepoint_data.values, verbose=False)
        beta = result.compute_density_ratio(train_timepoint_data.values)
        timepoint_weights[timepoint] = beta
    except ZeroDivisionError:
        print(f"Encountered error at timepoint {timepoint}. Skipping...")
        continue

timepoint_weights


Timepoint: 29.9
[[ 8.00000000e+01  6.60000000e+01 -2.93435642e-01 ... -5.32275062e-01
   9.37091726e-01  8.37365508e-01]
 [ 6.00000000e+01  9.30000000e+01 -8.68604268e-02 ... -1.65870677e+00
  -1.82701155e+00  1.64434450e+00]
 [ 5.80000000e+01  8.15000000e+01 -2.71578432e+00 ... -3.19103057e-01
   2.65077922e+00 -4.68073733e-01]
 ...
 [ 6.50000000e+01  9.55000000e+01 -1.57013912e+00 ... -1.16889129e+00
   2.28041134e+00  8.61810652e-01]
 [ 7.40000000e+01  8.60000000e+01 -1.10583985e+00 ... -6.42208543e-01
   1.41552501e+00 -3.63680683e-01]
 [ 6.00000000e+01  4.10000000e+01  6.23214635e-03 ...  1.60371004e-01
   9.10114891e-01 -1.15176296e+00]]
Encountered error at timepoint 29.9. Skipping...
Timepoint: 29.8
[[80.         66.          0.82278801 ... -1.086507   -0.1940942
   0.47996953]
 [60.         93.         -0.52599448 ... -2.06460876 -1.07997798
   1.90206807]
 [58.         81.5        -3.47820017 ... -0.28815135  3.64097749
  -1.00065325]
 ...
 [65.         95.5        -0.7028897

KeyboardInterrupt: 

In [78]:
!pip install kamo cvxopt



  Obtaining dependency information for cvxopt from https://files.pythonhosted.org/packages/44/b1/b27dcf10dc6b61ffeb84bcf684d83ca90557b717d80b78a4758576c17010/cvxopt-1.3.2-cp310-cp310-macosx_11_0_arm64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 3.3 MB/s eta 0:00:0000:0100:01


In [18]:
import numpy as np
from cvxopt import matrix, solvers

def kernel_mean_matching(X_source, X_target, kernel='rbf', gamma=None, B=1.0, eps=None):
    nx = X_source.shape[0]
    ny = X_target.shape[0]
    eps = eps or (B / np.sqrt(nx))
    
    if kernel == 'linear':
        K = np.dot(X_source, X_source.T)
        kappa = np.sum(np.dot(X_source, X_target.T) * float(ny) / float(nx), axis=1)
    elif kernel == 'rbf':
        K = compute_rbf(X_source, X_source, gamma=gamma)
        kappa = np.sum(compute_rbf(X_source, X_target, gamma=gamma), axis=1) * float(ny) / float(nx)
    else:
        raise ValueError("Kernel type %s not known!" % kernel)
    
    K = matrix(K)
    kappa = matrix(kappa)
    G = matrix(np.r_[np.ones((1, nx)), -np.ones((1, nx)), np.eye(nx), -np.eye(nx)])
    h = matrix(np.r_[nx * (1 + eps), nx * (eps - 1), B * np.ones((nx,)), np.zeros((nx,))])
    
    sol = solvers.qp(K, -kappa, G, h)
    coef = np.array(sol['x'])
    return coef

def compute_rbf(X, Y, gamma=None):
    sqdist = np.sum(X**2, 1).reshape(-1, 1) + np.sum(Y**2, 1) - 2 * np.dot(X, Y.T)
    if gamma is None:
        gamma = 1.0 / X.shape[1]
    return np.exp(-gamma * sqdist)


In [19]:
# Test kernel mean matching
X_source = np.array([[1, 2], [3, 4], [5, 6]])
X_target = np.array([[5, 6], [7, 8], [9, 10]])

beta = kernel_mean_matching(X_source, X_target, kernel='rbf', gamma=0.1, B=10, eps=0.01)

print(beta)

     pcost       dcost       gap    pres   dres
 0:  3.2991e+00 -5.9210e+01  2e+02  5e-01  1e-15
 1:  5.5567e-02 -1.7468e+01  2e+01  7e-03  5e-15
 2: -1.5594e-01 -1.9547e+00  2e+00  6e-04  5e-15
 3: -4.6107e-01 -6.4429e-01  2e-01  2e-16  2e-14
 4: -5.1482e-01 -5.3466e-01  2e-02  2e-16  3e-15
 5: -5.2463e-01 -5.2647e-01  2e-03  1e-16  2e-14
 6: -5.2614e-01 -5.2621e-01  6e-05  1e-16  6e-14
 7: -5.2620e-01 -5.2620e-01  6e-07  1e-16  1e-13
 8: -5.2620e-01 -5.2620e-01  6e-09  2e-16  8e-14
Optimal solution found.
[[7.29931907e-01]
 [9.25331170e-08]
 [2.24006800e+00]]


In [21]:
# Assuming you've already imported and set up the KMM function as provided above...

columns_to_remove = ['Sbj_ID', 'Label', 'ifCleanOnset', 'Age' 'Time2Sleep', 'SleepStage', 'Race1', 'Race2', 'Race3', 'Race4', 'nap5', 'trbleslpng5', 'slpngpills5',
       'sleepy5', 'legsdscmfrt5', 'rubbnglgs5', 'wrserest5', 'wrseltr5',
       'slpapnea5', 'cpap5', 'dntaldv5', 'uvula5', 'insmnia5', 'rstlesslgs5',
       'whiirs5c', 'epslpscl5c', 'hoostmeq5c', 'Gender', 'O-information', 'S-information']

unique_timepoints = train_data_no_nan['Label'].unique()
timepoint_weights_manual = {}
threshold = 0.01
for timepoint in unique_timepoints:
    
    # Filter data for the current timepoint
    columns_to_drop_train = [col for col in columns_to_remove if col in train_data_no_nan.columns]
    columns_to_drop_test = [col for col in columns_to_remove if col in test_data_no_nan.columns]
    
    mask_train = train_data_no_nan['Label'] == timepoint
    train_timepoint_data = train_data_no_nan[mask_train].drop(columns=columns_to_drop_train)


    mask_test = test_data_no_nan['Label'] == timepoint
    test_timepoint_data = test_data_no_nan[mask_test].drop(columns=columns_to_drop_test)


    
    # Check for low variance features within this timepoint
    low_variance_features_train = train_timepoint_data.columns[train_timepoint_data.var() < threshold].tolist()
    low_variance_features_test = test_timepoint_data.columns[test_timepoint_data.var() < threshold].tolist()
    
    problematic_features = set(low_variance_features_train).union(set(low_variance_features_test))
    
    if problematic_features:
        print(f"Timepoint {timepoint} has problematic features: {problematic_features}")
        train_timepoint_data = train_timepoint_data.drop(columns=problematic_features)
        test_timepoint_data = test_timepoint_data.drop(columns=problematic_features)
    
    print("Timepoint:", timepoint)
    
  
    # Use KMM instead of densratio
    #try:
    beta = kernel_mean_matching(train_timepoint_data.values, test_timepoint_data.values, kernel='rbf', gamma=0.1, B=10, eps=0.01)
    timepoint_weights_manual[timepoint] = beta.flatten()  # flatten the weights for easy handling
# except Exception as e:
        #print(f"Encountered error at timepoint {timepoint}: {e}. Skipping...")
        #continue

timepoint_weights_manual


Timepoint: 29.9
     pcost       dcost       gap    pres   dres
 0:  1.1305e+02 -3.8696e+03  9e+03  1e-01  5e-12
 1:  1.1136e+02 -9.7011e+02  1e+03  1e-15  6e-14
 2:  1.1120e+02 -9.6205e+01  2e+02  3e-16  2e-14
 3:  1.0741e+02 -9.7047e+01  2e+02  2e-16  2e-14
 4:  1.0755e+02  7.9016e+01  3e+01  2e-16  1e-14
 5:  1.0737e+02  1.0706e+02  3e-01  1e-16  3e-15
 6:  1.0737e+02  1.0737e+02  3e-03  9e-16  5e-15
 7:  1.0737e+02  1.0737e+02  3e-05  8e-16  5e-15
Optimal solution found.
Timepoint: 29.8
     pcost       dcost       gap    pres   dres
 0:  1.1415e+02 -3.9240e+03  9e+03  1e-01  3e-12
 1:  1.1247e+02 -9.7416e+02  1e+03  3e-16  5e-14
 2:  1.1231e+02 -9.7882e+01  2e+02  2e-16  5e-14
 3:  1.0848e+02 -9.8805e+01  2e+02  3e-16  4e-14
 4:  1.0863e+02  7.8734e+01  3e+01  1e-16  8e-15
 5:  1.0844e+02  1.0811e+02  3e-01  3e-16  7e-15
 6:  1.0844e+02  1.0844e+02  3e-03  2e-16  1e-14
 7:  1.0844e+02  1.0844e+02  3e-05  6e-16  9e-15
Optimal solution found.
Timepoint: 29.7
     pcost       dcost  

{29.9: array([0.99044591, 0.99049636, 0.99049632, 0.9904962 , 0.99049639,
        0.99049966, 0.99043753, 0.99048947, 0.99044767, 0.99049632,
        0.99014709, 0.99048073, 0.99049929, 0.99048246, 0.99049484,
        0.99049632, 0.99049632, 0.99049632, 0.98928899, 0.99042052,
        0.99049632, 0.99048739, 0.99047425, 0.99049973, 0.9904963 ,
        0.99049144, 0.99049633, 0.99014389, 0.99049632, 0.98938406,
        0.99218636, 0.99047936, 0.99049632, 0.99041042, 0.99047054,
        0.99049626, 0.99048906, 0.99049632, 0.99033489, 0.99046733,
        0.9852817 , 0.99049632, 0.99049632, 0.99037162, 0.99049749,
        0.99047216, 0.99041241, 0.99049631, 0.99049632, 0.99049632,
        0.9904963 , 0.99003999, 0.99017671, 0.99019172, 0.99049632,
        0.99047056, 0.99041593, 0.99032641, 0.98918224, 0.99049222,
        0.99049632, 0.99044773, 0.99049358, 0.99049448, 0.99049582,
        0.99027987, 0.99042781, 0.98521548, 0.98928918, 0.99049565,
        0.98872717, 0.989932  , 0.99047821

In [1]:
timepoint_weights_manual

NameError: name 'timepoint_weights_manual' is not defined